In [ ]:
# %%
## Setup
from openai import OpenAI
import json
import csv
import os

os.environ["OPENAI_API_KEY"] = ""
client = OpenAI()

In [ ]:


# %%
## Batch Options

# Datasets
dataset = "dataset-100.csv"
# dataset = "dataset-3.csv"

# Jobs
jobName = "One 100 Row Job"
# jobName = "Sample 3 Row Job"

# Json Format
string1 = "{\"custom_id\": \"request-" 
string2 = "\", \"method\": \"POST\", \"url\": \"/v1/chat/completions\", \"body\": {\"model\": \"gpt-3.5-turbo-0125\", \"messages\": [{\"role\": \"system\", \"content\": \"You are an sentiment analyst.\"},{\"role\": \"user\", \"content\": \""
string3 = "\"}],\"max_tokens\": 3000}}"

In [ ]:
## Create a jsonl file in OpenAI's Format

with open(dataset, mode="r", newline="", encoding="utf-8") as csv_file:
    with open("data.jsonl", mode="w", encoding="utf-8") as jsonl_file:
        
        reader = csv.reader(csv_file)
        
        for row in reader:
            sampleString = string1
            
            sampleString = sampleString + row[0]
            sampleString = sampleString + string2
            
            sampleString = sampleString + row[1].replace('"', '\\"')
            sampleString = sampleString + string3
            
            jsonl_file.write(sampleString + "\n") 

In [ ]:
## Upload Batch Input File
batch_input_file = client.files.create(
    file=open("data.jsonl", "rb"),
    purpose="batch"
)

response_dict = batch_input_file.model_dump()

with open("batch-upload.json", "w", encoding="utf-8") as json_file:
    json.dump(response_dict, json_file, indent=4)


# print(batch_input_file)

In [ ]:
## Creating the Batch Job
batch_input_file_id = batch_input_file.id

batch_create_file = client.batches.create(
    input_file_id=batch_input_file_id,
    endpoint="/v1/chat/completions",
    completion_window="24h",
    metadata={
        "description": jobName
    }
)

response_dict = batch_create_file.model_dump()

with open("batch-created.json", "w", encoding="utf-8") as json_file:
    json.dump(response_dict, json_file, indent=4)
